# 對話式 RAG 系統

本範例展示：
1. **第1個儲存格**：載入向量資料庫並建立檢索器
2. **第2個儲存格**：建立對話式 RAG Chain（含歷史記憶）

學習目標：理解如何建立具有上下文記憶的多輪對話 RAG 系統

In [ ]:
# 第1個儲存格：載入向量資料庫並建立檢索器

import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 從 .env 載入環境變數
load_dotenv()

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

# 建立用於查詢向量存儲的檢索器
# search_type="similarity": 使用相似度搜尋
# k=3: 返回最相關的 3 個文件
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

print("向量資料庫和檢索器已準備就緒")
print(f"資料庫位置: {persistent_directory}")
print(f"檢索器設定: 相似度搜尋，返回 3 個文件")

In [ ]:
# 第2個儲存格：建立對話式 RAG Chain（含歷史記憶）

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama

# 建立 ChatOllama 模型
llm = ChatOllama(model="llama3.2")

# 情境化問題提示
# 此提示幫助 AI 理解應根據聊天歷史重新表述問題，使其成為獨立的問題
contextualize_q_system_prompt = (
    "給定聊天歷史和最新的使用者問題，"
    "該問題可能引用聊天歷史中的上下文，"
    "請制定一個獨立的問題，可以在沒有聊天歷史的情況下理解。"
    "不要回答問題，只需在需要時重新表述，否則按原樣返回。"
)

# 建立用於情境化問題的提示模板
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [ 
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 建立具有歷史意識的檢索器
# 這使用 LLM 幫助根據聊天歷史重新表述問題
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# 回答問題提示
# 此提示幫助 AI 理解應根據檢索到的上下文提供簡潔的答案
qa_system_prompt = (
    "你是一個問答任務的助手。使用"
    "以下檢索到的上下文片段來回答"
    "問題。如果你不知道答案，就說你"
    "不知道。最多使用三個句子並保持答案"
    "簡潔。"
    "\n\n"
    "{context}"
)

# 建立用於回答問題的提示模板
qa_prompt = ChatPromptTemplate.from_messages(
    [ 
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 建立用於問答的文件組合鏈
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# 建立結合具有歷史意識的檢索器和問答鏈的檢索鏈
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

print("對話式 RAG Chain 已建立完成")
print("\n--- RAG Chain 架構 ---")
print("1. 歷史意識檢索器：根據聊天歷史重新表述問題")
print("2. 向量檢索：找出相關文件")
print("3. 問答鏈：基於文件和歷史生成答案")

# 模擬持續聊天的函數
def continual_chat():
    print("\n開始與 AI 聊天！輸入 'exit' 結束對話。")
    chat_history = []  # 在此收集聊天歷史（訊息序列）
    while True:
        query = input("你：")
        if query.lower() == "exit":
            break
        # 透過檢索鏈處理使用者的查詢
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # 顯示 AI 的回應
        print(f"AI：{result['answer']}")
        # 更新聊天歷史
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result["answer"]))

# 啟動持續聊天
continual_chat()

In [ ]:
# 第3個儲存格：逐步執行對話式 RAG Chain

# 由於第二個儲存格的 continual_chat() 函數會導致程式卡在輸入迴圈，
# 我們在此建立一個新的儲存格，以手動、逐步的方式來執行對話，
# 這樣可以更清楚地展示每一步的輸入和輸出。

from langchain_core.messages import HumanMessage, AIMessage

# 初始化聊天歷史
chat_history = []

# --- 第 1 次對話 ---
print("\n--- 第一次對話 ---")
question1 = "洗衣機的建議安裝地點在哪裡？"
print(f"使用者問題: {question1}")

# 執行 RAG Chain
# 我們傳遞問題和當前的聊天歷史（目前是空的）
result1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})

print(f"\nAI 回應: {result1['answer']}")

# 將第一次對話的問與答加入聊天歷史
chat_history.extend([HumanMessage(content=question1), AIMessage(content=result1["answer"])])

print("\n--- 檢視第一次對話的上下文 ---")
# 顯示檢索到的文件，以了解 AI 回應的依據
for i, doc in enumerate(result1['context'], 1):
    print(f"文件 {i}: {doc.page_content[:100]}...")

# --- 第 2 次對話 ---
print("\n\n--- 第二次對話（有歷史紀錄） ---")
question2 = "那裡有提到電源要注意什麼嗎？"
print(f"使用者問題: {question2}")

# 再次執行 RAG Chain
# 這次，我們傳遞了包含第一次對話的聊天歷史
result2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})

print(f"\nAI 回應: {result2['answer']}")

# 更新聊天歷史
chat_history.extend([HumanMessage(content=question2), AIMessage(content=result2["answer"])])

print("\n--- 檢視第二次對話的上下文 ---")
# 觀察 `history_aware_retriever` 如何根據歷史重寫問題並找到相關文件
for i, doc in enumerate(result2['context'], 1):
    print(f"文件 {i}: {doc.page_content[:100]}...")

print("\n\n--- 最終聊天歷史 ---")
print(chat_history)